## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-10-38-23 +0000,bbc,Flights in Afghanistan grounded after internet...,https://www.bbc.com/news/articles/cdxqdy5nrlqo...
1,2025-09-30-10-38-23 +0000,nypost,Prosecutors want Sean ‘Diddy’ Combs to get 11 ...,https://nypost.com/2025/09/30/us-news/prosecut...
2,2025-09-30-10-35-05 +0000,nyt,What We Know About ChatGPT’s New Parental Cont...,https://www.nytimes.com/2025/09/30/technology/...
3,2025-09-30-10-29-55 +0000,nypost,Don Lemon claims ‘white men’ are ‘broken’ and ...,https://nypost.com/2025/09/30/us-news/don-lemo...
4,2025-09-30-10-29-10 +0000,nyt,"With New U.S. Proposal to End Gaza War, a Rare...",https://www.nytimes.com/2025/09/30/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,58
4,shutdown,19
202,government,18
12,new,18
30,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,147
184,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,142
159,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,118
70,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,118
141,2025-09-29-23-27-45 +0000,nyt,Trump and Netanyahu Tell Hamas to Accept Gaza ...,https://www.nytimes.com/2025/09/29/us/politics...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,147,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
184,84,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
28,68,2025-09-30-09-30-00 +0000,wsj,With roughly a month left in the New York City...,https://www.wsj.com/politics/elections/cuomo-p...
135,44,2025-09-30-00-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/you-cant-r...
27,43,2025-09-30-09-30-00 +0000,wsj,A government shutdown would likely mean federa...,https://www.wsj.com/politics/policy/how-govern...
219,40,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
63,35,2025-09-30-07-40-13 +0000,nypost,Woman disrobes at California school board meet...,https://nypost.com/2025/09/30/us-news/woman-di...
211,35,2025-09-29-19-32-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
253,33,2025-09-29-16-49-04 +0000,cbc,Police continue to search for motive in deadly...,https://www.cbc.ca/news/canada/windsor/deadly-...
193,32,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
